# `CLOUD.jl` - 2D linear advection example

In [27]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [28]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 0.5/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [29]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm()
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [30]:
reference_approximation = ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50,
    operator_algorithm=DefaultOperatorAlgorithm())

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [36]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            853ms /  89.6%            512MiB /  97.2%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual         20.4k    680ms   88.9%  33.3μs    436MiB   87.7%  21.9KiB
   apply operators      20.4k    302ms   39.5%  14.8μs    246MiB   49.5%  12.4KiB
     volume terms       20.4k    106ms   13.9%  5.19μs    117MiB   23.4%  5.84KiB
     facet terms        20.4k   88.7ms   11.6%  4.35μs   95.6MiB   19.2%  4.80KiB
     mass matrix solve  20.4k   32.0ms    4.2%  1.57μs   10.0MiB    2.0%     512B
   gather ext state     20.4k   74.2ms    9.7%  3.64μs   67.3MiB   13.5%  3.38KiB
   eval nodal s

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [32]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.005587844498469991
Conservation (initial/final/diff):
[8.023589626732219e-8][8.023589542077714e-8][-8.465450562766819e-16]
Energy (initial/final/diff):
[0.12499874137052301][0.12470788892688732][-0.00029085244363569185]


Write output to VTK file

In [33]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));